In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import os
import csv
import matplotlib.pyplot as plt
data_drive = "drive/MyDrive/ebay_data_2023"

Mounted at /content/drive


In [84]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
train_df = pd.read_csv(data_drive + "/Train_Tagged_Titles.tsv", sep="\t", dtype=str, keep_default_na=False, na_values=[""], quoting=csv.QUOTE_NONE)
old_tag = train_df.Tag.copy()
train_df

,Record Number,Title,Token,Tag
0,1,Supreme Nike SB Dunk High By any Means Red US1...,Supreme,Modell
1,1,Supreme Nike SB Dunk High By any Means Red US1...,Nike,Marke
2,1,Supreme Nike SB Dunk High By any Means Red US1...,SB,Produktlinie
3,1,Supreme Nike SB Dunk High By any Means Red US1...,Dunk,NaN
4,1,Supreme Nike SB Dunk High By any Means Red US1...,High,Schuhschaft-Typ
...,...,...,...,...
55178,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,Sportschuhe,Produktart
55179,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,Wanderschuh,NaN
55180,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,Big,No Tag
55181,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,Size,No Tag


All augmented rows will have record numbers 5000+ (correpond to IDs in the training data set fed to the model) but may have Title "AUGMENT" as this information is discarded.

In [327]:
tag_groups_train = train_df.groupby("Tag").apply(lambda x: np.unique(x.Token.values))
random_adds = tag_groups_train.loc[["No Tag", "Obscure", "EU-Schuhgröße", "Marke", "Farbe", "Produktart", "US-Schuhgröße", "Modell", "UK-Schuhgröße", "Produktlinie", "Obermaterial", "Abteilung", "Stil", "Akzente"]]
random_adds

Tag
No Tag           [!, !!, !!!, !!!!, !!!!!, !!!!!!, ", #, #I, #O...
Obscure          ['s, ,, ., 1SportSchuhe, 206340, 31Low, 5, 574...
EU-Schuhgröße    [,40, ,44, .35, .36, .36,5, .37, .37,39,40, .3...
Marke            [&, 0, 4F, 7, 77, A, ADIDAS, ALDI, ALEJANDRO, ...
Farbe            [AUTUMN, Aero, All, Amber, Anthracite, Anthraz...
Produktart       [Aktivschuhe, Alltags-Schuhe, Alltagsschuhe, A...
US-Schuhgröße    [.9, 10, 10,5, 10.5, 11, 11,5, 11.5, 12, 12.5,...
Modell           ['07, -, 03, 05, 07, 1, 1.3, 1.4, 10, 106, 106...
UK-Schuhgröße    [.10,5, .6, 10, 10,5, 10.5, 11, 12.5, 13, 5, 5...
Produktlinie     [192363, 3Lab5, A-ZX, ACG, ADI, ADVENTURE, AF1...
Obermaterial     [Baumwolle, Bio-Baumwolle, CALFHAIR, CHEVRON, ...
Abteilung        [Baby, DAMEN, DAMENSCHUHE, DONNA, Dam, Dame, D...
Stil             [Ballerina, Ballerinas, Baskets, Budapester, F...
Akzente          [Allover, Cut, Fransen, GLITZER, Glanz, Glitte...
dtype: object

,Record Number,Title,Token,Tag
28,3,♥ MICHAEL KORS Sneaker Gr 39 ♥,♥,No Tag
29,3,♥ MICHAEL KORS Sneaker Gr 39 ♥,MICHAEL,Marke
30,3,♥ MICHAEL KORS Sneaker Gr 39 ♥,KORS,NaN
31,3,♥ MICHAEL KORS Sneaker Gr 39 ♥,Sneaker,Stil
32,3,♥ MICHAEL KORS Sneaker Gr 39 ♥,Gr,No Tag
33,3,♥ MICHAEL KORS Sneaker Gr 39 ♥,39,EU-Schuhgröße
34,3,♥ MICHAEL KORS Sneaker Gr 39 ♥,♥,No Tag


In [36]:
# 1 Adding shoe sizes

size_keys = ["US-Schuhgröße", "EU-Schuhgröße", "UK-Schuhgröße"]
sizes_to_add = {"US-Schuhgröße":[], "EU-Schuhgröße":[], "UK-Schuhgröße":[]}

eu_sz = np.arange(38,46).astype(str)
eu_add = eu_sz.copy()
for size_str in eu_sz:
    eu_add = np.append(eu_add, size_str+",5")
    eu_add = np.append(eu_add, "EU" + size_str)
    eu_add = np.append(eu_add, "." + size_str)
    eu_add = np.append(eu_add, "." + size_str+",5")
    eu_add = np.append(eu_add, "Gr." + size_str)
    eu_add = np.append(eu_add, "Gr" + size_str)
    eu_add = np.append(eu_add, "GR" + size_str)
    eu_add = np.append(eu_add, "EUR_" + size_str)

eu_add = np.append(eu_add, "/".join(eu_sz))
eu_add = np.append(eu_add, ".".join(eu_sz))
eu_add = np.append(eu_add, ["38-45", "38-44", "38-43", "39-45", "39-44", "39-43"])

sizes_to_add["EU-Schuhgröße"].extend(eu_add)

uk_sz = np.arange(5,16).astype(str)
uk_add = uk_sz.copy()
for size_str in uk_sz:
    uk_add = np.append(uk_add, size_str+",5")
    uk_add = np.append(uk_add, "UK" + size_str)
    uk_add = np.append(uk_add, size_str)
    uk_add = np.append(uk_add, size_str+",5")
    uk_add = np.append(uk_add, size_str+".5")
    uk_add = np.append(uk_add, "UK_" + size_str)

uk_add = np.append(uk_add, ["5-15", "5-14", "5-13", "5-12", "6-15", "6-14", "6-13", "6-12", "7-15", "7-14", "7-13", "7-12"])

sizes_to_add["UK-Schuhgröße"].extend(uk_add)

us_sz = np.arange(6,16).astype(str)
us_add = us_sz.copy()
for size_str in us_sz:
    us_add = np.append(us_add, size_str+",5")
    us_add = np.append(us_add, "UK" + size_str)
    us_add = np.append(us_add, size_str)
    us_add = np.append(us_add, size_str+",5")
    us_add = np.append(us_add, size_str+".5")
    us_add = np.append(us_add, "UK_" + size_str)

us_add = np.append(us_add, ["6-15", "6-14", "6-13", "6-12", "7-15", "7-14", "7-13", "7-12", "8-15", "8-14", "8-13", "8-12"])

sizes_to_add["US-Schuhgröße"].extend(us_add)

In [103]:
def get_index(series, word):
    return ((series == word).astype(int)*series.index).sum()

In [403]:
def replace_row(df, line, i):
    df2 = pd.concat([df.loc[:i-1, :], line, df.loc[i+1:, :]]).reset_index(drop=True)
    return df2.sort_index()

In [396]:
def insert_row(df, line, i):
    df2 = pd.concat([df.loc[:i, :], line, df.loc[i:, :]]).reset_index(drop=True)
    return df2.sort_index()

In [460]:
def augment(y):
    y.Title = "AUGMENT"

    prev = -1
    add_rows = y.copy()
    brand_choice = None
    row_change = None
    for row in y.iterrows():
        ind = row[0]
        x = row[1]
        if x["Tag"] in size_keys:
            x["Token"] = np.random.choice(sizes_to_add[x["Tag"]])
        if x["Token"] in ["Sneaker", "sneaker"]:
            x["Token"] = np.random.choice(["sneakers", "Sneakers", "sneaker", "Snaker"])
        if prev == "Marke" and not pd.isna(x["Tag"]):
            if np.random.rand() < 0.3:
                brand_choice = np.random.choice(["Nike", "Vans", "Converse"])
                row_change = ind-1
        prev = x.Tag

    if row_change and brand_choice:
        brand_line = pd.DataFrame({"Title":"AUGMENT", "Token":brand_choice, "Tag":"Marke"}, index = [row_change])
        y = replace_row(y, brand_line, row_change)

    in_grobe = get_index(y.Tag, "EU-Schuhgröße")-1
    in_grobe = in_grobe if in_grobe != -1 else (y.index[-1]+1)
    Grline = pd.DataFrame({"Title":"AUGMENT", "Token":"gr", "Tag":"No Tag"}, index = [in_grobe])
    grobeline = pd.DataFrame({"Title":"AUGMENT", "Token":"größe", "Tag":"No Tag"}, index = [in_grobe])
    size_lines = list([Grline, grobeline])
    size_insert_line = size_lines[np.random.choice(len(size_lines))]
    if in_grobe != y.index[-1]+1:
        y = replace_row(y, size_insert_line, in_grobe)
    else:
       if np.random.rand() < 0.2:
          y = insert_row(y, size_insert_line, y.index[-1]+1)

    if np.random.rand() < 0.3:
        in_eu = get_index(y.Tag, "EU-Schuhgröße")-1
        in_eu = in_eu if in_eu != -1 else (y.index[-1]+1)
        eu_line = pd.DataFrame({"Title":"AUGMENT", "Token":"eu", "Tag":"EU-Schuhgröße"}, index = [in_eu])
        if in_grobe != y.index[-1]+1:
            y = insert_row(y, eu_line, in_eu)
        else:
          if np.random.rand() < 0.2:
              y = insert_row(y, eu_line, y.index[-1]+1)

        in_us = get_index(y.Tag, "US-Schuhgröße")-1
        in_us = in_us if in_us != -1 else (y.index[-1]+1)
        us_line = pd.DataFrame({"Title":"AUGMENT", "Token":"us", "Tag":"US-Schuhgröße"}, index = [in_us])
        if in_grobe != y.index[-1]+1:
            y = insert_row(y, us_line, in_us)
        else:
          if np.random.rand() < 0.2:
              y = insert_row(y, us_line, y.index[-1]+1)

        in_uk = get_index(y.Tag, "UK-Schuhgröße")-1
        in_uk = in_uk if in_uk != -1 else (y.index[-1]+1)
        uk_line = pd.DataFrame({"Title":"AUGMENT", "Token":"uk", "Tag":"UK-Schuhgröße"}, index = [in_uk])
        if in_grobe != y.index[-1]+1:
            y = insert_row(y, uk_line, in_uk)
        else:
          if np.random.rand() < 0.2:
              y = insert_row(y, uk_line, y.index[-1]+1)


    # with probability add in "schwarz" or "damen"
    if np.random.rand() < 0.3:
        schwarz_row = pd.DataFrame({"Title":"AUGMENT", "Token":"damen", "Tag":"Abteilung"}, index = [y.index[-1]+1])
        y = insert_row(y, schwarz_row, y.index[-1])

    if np.random.rand() < 0.5:
        schwarz_row = pd.DataFrame({"Title":"AUGMENT", "Token":"schwarz", "Tag":"Farbe"}, index = [y.index[-1]+1])
        y = insert_row(y, schwarz_row, y.index[-1])

    # adding random entities
    num_to_add = np.random.choice(np.arange(0,3))
    for _ in range(num_to_add):
        rand_ind = np.random.choice(np.arange(0,len(random_adds)))
        rand_add_tup = random_adds.index[rand_ind], np.random.choice(random_adds.iloc[rand_ind])
        rand_pos = np.random.choice(np.append(y.index, y.index[-1]+1))
        random_row = pd.DataFrame({"Title":"AUGMENT", "Token":rand_add_tup[1], "Tag":rand_add_tup[0]}, index = [rand_pos])
        y = insert_row(y, random_row, rand_pos)

    y["Record Number"] = y["Record Number"].ffill()

    return y

In [497]:
train_df["Record Number"] = train_df["Record Number"].astype(int)

In [499]:
out = train_df.copy()
for i in range(1, 5001):
    record = train_df.loc[train_df['Record Number'] == i, :]
    #augment data twice, randomly
    out1 = record.groupby("Record Number").apply(augment)
    out2 = record.groupby("Record Number").apply(augment)

    out1["Record Number"] += 5000
    out2["Record Number"] += 10000
    out = pd.concat([out, out1, out2])
out

,Record Number,Title,Token,Tag
0,1.0,Supreme Nike SB Dunk High By any Means Red US1...,Supreme,Modell
1,1.0,Supreme Nike SB Dunk High By any Means Red US1...,Nike,Marke
2,1.0,Supreme Nike SB Dunk High By any Means Red US1...,SB,Produktlinie
3,1.0,Supreme Nike SB Dunk High By any Means Red US1...,Dunk,NaN
4,1.0,Supreme Nike SB Dunk High By any Means Red US1...,High,Schuhschaft-Typ
...,...,...,...,...
"(5000, 7)",15000.0,AUGMENT,Sportschuhe,Produktart
"(5000, 8)",15000.0,AUGMENT,Wanderschuh,NaN
"(5000, 9)",15000.0,AUGMENT,Big,No Tag
"(5000, 10)",15000.0,AUGMENT,Size,No Tag


In [502]:
out = out.reset_index(drop=True)
out["Record Number"] = out["Record Number"].astype(str)
# out["Record Number"] = out["Record Number"].ffill().astype(int) + 5000

In [503]:
out

,Record Number,Title,Token,Tag
0,1.0,Supreme Nike SB Dunk High By any Means Red US1...,Supreme,Modell
1,1.0,Supreme Nike SB Dunk High By any Means Red US1...,Nike,Marke
2,1.0,Supreme Nike SB Dunk High By any Means Red US1...,SB,Produktlinie
3,1.0,Supreme Nike SB Dunk High By any Means Red US1...,Dunk,NaN
4,1.0,Supreme Nike SB Dunk High By any Means Red US1...,High,Schuhschaft-Typ
...,...,...,...,...
209365,15000.0,AUGMENT,Sportschuhe,Produktart
209366,15000.0,AUGMENT,Wanderschuh,NaN
209367,15000.0,AUGMENT,Big,No Tag
209368,15000.0,AUGMENT,Size,No Tag


In [506]:
filename = "augmented_data_3.csv"
out.to_csv(data_drive + filename, sep="\t", na_rep = "null")